In [35]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from scipy.optimize import minimize, LinearConstraint, linprog
import pandas as pd
import sympy

def equal_conversion(A, b):
    A_tilde = np.hstack((A, np.identity(A.shape[0])))
    return A_tilde, b

def find_z(A, b, x_dim):
    n = A.shape[0]
    d = A.shape[1]
    for i in range(x_dim, d):
        A_ub = A.T * -1
        b_ub = np.zeros(d)
        
        A_eq = np.vstack((A.T[0:x_dim], A.T[i], b.T))

        b_eq = np.zeros(x_dim)
        b_eq = np.append(b_eq, [1,0])
        
        c = np.ones(n)
        
        res = linprog(
            c = c,
            A_ub = A_ub,
            b_ub = b_ub,
            A_eq = A_eq,
            b_eq = b_eq
        )
        if res["success"]:
            return np.matmul(A.T, res.x)
        
    return False 

def facial_reduction(z):
    d = z.shape[0]
    z = np.round(z, 3)
    indices = (z <= 0).nonzero()[0]
    matrix = None
    
    for val in indices:
        arr = np.eye(1, d, val)
        matrix = arr if matrix is None else np.vstack((matrix, arr))
    
    return matrix.T

def entire_facial_reduction_step(A, b, x_dim):
    z = find_z(A, b, x_dim)
    if isinstance(z, bool):
        return (A, b)
    
    V = facial_reduction(z)

    AV = np.matmul(A, V)


    _, ind = sympy.Matrix(AV).T.rref()
    print(_)
    print(ind)
    proj = None
    for i in ind:
        arr = np.eye(1, A.shape[0], i)
        proj = arr if proj is None else np.vstack((proj, arr))
    newA = np.matmul(proj, AV)
    newB = np.matmul(proj, b)

    return entire_facial_reduction_step(newA, newB, x_dim)

def make_full_rank(mat):
    Q, R = np.linalg.qr(mat.T, mode = "complete")
    dim_diff = Q.shape[0] - R.shape[1]
    if dim_diff == 0:
        return mat
    
    new_mat = np.identity(Q.shape[0])[:,  Q.shape[0] - dim_diff:]
    
    R = np.hstack((R, new_mat))
    return np.matmul(Q, R).T

def reduce_sampling(M, b, delta_dim):
    M_inv = np.linalg.inv(M)
    row = M_inv.shape[0] - delta_dim
    gamma = M_inv.shape[0] - b.shape[0]
    C = np.matmul(M_inv[row:, :M_inv.shape[1] - gamma], b)
    D = -1 * M_inv[row:, M_inv.shape[1] - gamma:]
    return D, C

def reduce_problem(A, b):
    newA, newB = equal_conversion(A, b)
    A_tilde, b_tilde = entire_facial_reduction_step(newA, newB, A[1].shape[0])
    if A_tilde.shape == newA.shape and b.shape == b_tilde.shape:
        return (A, b)

    M = make_full_rank(A_tilde)
    
    reduced_A, reduced_b = reduce_sampling(M, b_tilde, A_tilde.shape[1] - A[1].shape[0])
    return reduced_A, reduced_b, b_tilde, M

In [135]:
A = np.array([[1, 0],[-1, 0],[0, 1],[0, -1]])
b = np.array([[2],[2],[0],[0]])

In [128]:
A = np.array([[1, 0, 0],[-1, 0, 0],[0, 1, 0],[0, -1, 0],[0, 0, 1],[0, 0, -1]])
b = np.array([[1],[1],[1],[1],[0],[0]])

In [61]:
A = np.array([[-1, 0], [0, -1], [1, 1], [-1, -1]])
b = np.array([[0],[0],[1],[-1]])

In [35]:
A = np.matrix([[-1, 0], [0, -1],[1, 1]])
b = np.matrix([[0],[0],[1]])

In [37]:
A = np.matrix([[1, 0], [-1, 0],[0, 1], [0, -1]])
b = np.matrix([[1],[0],[1],[0]])

In [12]:
A = np.array([[1, 0, 0],[-1, 0, 0],[0, 1, 0],[0, -1, 0],[0, 0, 1],[0, 0, -1]])
b = np.array([[1],[1],[0],[0],[0],[0]])

In [36]:
reduced_A, reduced_B, pB, M = reduce_problem(A, b)

Matrix([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, -1.00000000000000, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
(0, 1, 2, 4, 5)
Matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, -1.00000000000000], [0, 0, 0, 0, 0]])
(0, 1, 2, 3)


In [7]:
reduced_A

array([[ 0.57735027],
       [-0.57735027]])

In [8]:
reduced_B

array([[1.],
       [1.]])

In [9]:
pB

array([[1.],
       [1.],
       [0.],
       [0.]])

In [10]:
M

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.66533454e-16,  1.00000000e+00],
       [-7.85046229e-17,  1.00000000e+00,  0.00000000e+00,
         7.85046229e-17,  2.22044605e-16],
       [-6.76653964e-17, -2.18846843e-16,  1.00000000e+00,
         6.76653964e-17, -6.76653964e-17],
       [ 5.77350269e-01, -1.11022302e-16, -1.11022302e-16,
        -5.77350269e-01,  5.77350269e-01]])

In [134]:
mat = np.array([[1, -1, 0], [0, 0, 1], [0, 1, 0], [2, 2, 0]])

In [153]:
def old_find_z(A, b, x_dim):
    n = A.shape[0]
    d = A.shape[1]
    for i in range(x_dim, d):
        i = 4
        A_ub = A.T * -1
        b_ub = np.zeros(d)
        
        A_eq = np.vstack((A.T[0:x_dim], A.T[i], b.T))
        #A_eq = np.hstack((A_eq, np.expand_dims(np.zeros(A_eq.shape[0]), axis = 1)))
        
        
        b_eq = np.zeros(x_dim)
        b_eq = np.append(b_eq, [1,0])
        
        return A_eq, b_eq
        
        c = np.zeros(n + 1)
        c[n] = 1
        
        res = linprog(
            c = c,
            A_ub = A_ub,
            b_ub = b_ub,
            A_eq = A_eq,
            b_eq = b_eq
        )
        if res["success"]:
            return np.matmul(A.T, res.x)
        
    return False 

In [154]:
newA, newB = equal_conversion(A, b)
A_eq, b_eq = old_find_z(newA, newB, 2)